In [1]:
import backend
import json
initial_prompt = "Create a schema for a very simple twitter application"
schema_path = f'generated/schema.json'
schema_page_path= f'test/convex/schema.ts'
actions_path = 'generated/actions.json'
crud_path = 'generated/crud.ts'
backend_json = 'generated/backend.json'
backend_path = 'test/convex/backend.ts'
page_path = 'generated/page_schema.json'


In [ ]:
backend.write_to_file(backend.create_schema_structure(initial_prompt), schema_path)

In [2]:
schema = backend.read(schema_path)
backend.write_to_file(backend.create_actions(f"Create it for the following structure: \n<applicationStructure>\n{schema}\n</applicationStructure>"), actions_path)

Here is the generated schema for the Twitter-like application:

<jsonSchema>
{
  "actions_description": "The application will need actions to get a user's profile information, get a user's tweets, get the list of users a user is following, get the list of users following a user, post a new tweet, and follow/unfollow another user.",
  "query_actions_required": "Query actions will be needed to retrieve user profiles, user tweets, and follower/following lists.",
  "query_actions": [
    {
      "name": "getUserProfile",
      "where_used": "Used on a user's profile page to get their profile details",
      "docs": "Retrieves a user's profile information",
      "requires_auth": false,
      "arguments": [
        {
          "name": "username",
          "docs": "The username of the user whose profile to retrieve",
          "type": "v.string()"
        }
      ],
      "returns": "The user's profile information",
      "return_type": "User object",
      "workflow_steps": [
        {
   

In [3]:
# entirely compiler
schema = json.loads(backend.read(schema_path))
actions = backend.read(actions_path)
schema_page = backend.create_schema(schema, schema_page_path)
crud_page = backend.create_crud(schema, crud_path)
# faker_data = backend.create_faker_data_code(schema, 'generated/faker.ts')
actions_code = backend.create_actions_code(actions, schema_page, crud_page, backend_json)
actions_page = backend.create_actions_page(json.loads(actions), json.loads(actions_code),crud_page, backend_path)

Here is the code for the requested actions in the specified JSON schema format:

<jsonSchema>
{
  "getUserProfile": "const user = await getManyUsers(d, (user) => user.username == args.username).unique();\nreturn user;",
  "getUserTweets": "const tweets = await getManyTweets(d, (tweet) => tweet.userId == args.userId).collect();\nreturn tweets;",
  "getFollowing": "const follows = await getManyFollows(d, (follow) => follow.followerId == args.userId).collect();\nconst followedUserIds = follows.map(follow => follow.followedId);\nconst followedUsers = await Promise.all(followedUserIds.map(userId => getOneUsers(d, userId)));\nreturn followedUsers;",
  "getFollowers": "const follows = await getManyFollows(d, (follow) => follow.followedId == args.userId).collect();\nconst followerUserIds = follows.map(follow => follow.followerId);\nconst followerUsers = await Promise.all(followerUserIds.map(userId => getOneUsers(d, userId)));\nreturn followerUsers;",
  "postTweet": "const identity = await d.au

In [ ]:
# creating the page structure
schema = backend.read(schema_path)
actions = backend.read(actions_path)
backend.write_to_file(backend.create_page_structure("Task: " + initial_prompt + "\n\nHere is the database schema for the avaiable database objects you are to use: \n<Schemas>\n" + str(schema) + "\n</Schemas>\n\n Here are the backend queries and mutations that you can call: \n<backend>" + str(actions) + "\n</backend>\n\nNow, generate the page layout strictly in the json format described above." ), page_path)